------
## THE NET PROMOTER SCORE
Net Promoter Score, more commonly known as NPS, is a [customer success metric](https://en.wikipedia.org/wiki/Customer_success) that quantifies customer satisfaction. You've probably been asked to fill in surveys that contained some variation of the question "How likely are you to recommend this product to a friend?" and whose possible answers were 11 numbers on a scale from 0 to 10.



![](images/nps_question.png)




Customers who pick:

- Anything from 0 through 6 are called detractors;
- 7 or 8 are called passives;
- 9 or 10 are called promoters;

By proxy each of these categories are gauged as follows:

- `Detractors`: Unhappy customers who share their negative experience with the product or company; they may actively spread negative feedback.

- `Passives`: Customers who may be pleased with the product, but aren't actively promoting it and will quickly turn to the competition if a better opportunity presents itself.

- `Promoters`: Extremely satisfied customers who will act as brand ambassadors.

NPS is then calculated as the percentage of promoters minus the percentage of detractors.

![](images/nps_computation.png)

#### Categorize Function
The function, `categorize`, returns the respective category for each possible score:

   - Scores `0` through `6` should be categorized as `"Detractor"`;

   - Scores `7` and `8` should be categorized as `"Passive"`;

   - Scores `9` and `10` should be categorized as `"Promoter"`;

In [1]:
def categorize(score):
    """Returns NPS category"""
    if score in range(0,7):
        return "Detractor"
    elif score in (7, 8):
        return "Passive"
    elif score in (9,10):
        return "Promoter"
    return None

Note that NPS can range between -100% (if everyone is a detractor) and 100% (if everyone is a promoter).

Let's see a small example:

|Customer|Score|
|-------|-----|
|Rashid Nezhmetdinov|8|
|Vasily Smyslov|10|
|José Raúl Capablanca|9|
|Robert James Fischer|5|
|Emanuel Lasker|7|
|Aron Nimzowitsch|2|
|Mikhail Botvinnik|3|
|Mikhail Tal|1|

We have:

- 8 customers
- 2 promoters: Capablanca and Smyslov;
  - Therefore the percentage of promoters is 25%
- 2 passives: Nezhmetdinov and Lasker
- 4 detractors: Fischer, Nimzowitsch, Botvinnik and Tal
  - Therefore the percentage of detractors is 50%

The NPS is 25-50, or -25. Usually people do not include the percentage sign and just say that the NPS is -25, but it's simply an aesthetics choice.

Below is the computation of the net promoter score.

The data is stored in the file `nps_dataset.csv`.


#### Import Dependencies

In [ ]:
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns

Read in the dataset stored in the file `nps_dataset.csv` as a DataFrame

In [ ]:
# Read in the dataset as a DataFrame
df = pd.read_csv("inputData/nps_dataset.csv")
df.head(7)

Extract the dates, ids, and scores from the DataFrame and create a new DataFrame `df` with three columns:

- `event_date`: The date and time in which customers completed the survey
- `user_id`: A customer identifier
- `score`: Their answer to the question featured above

The dates pertain the year of 2020 only and the dataset has no missing values.

In [ ]:
# Extract the dates, ids, and scores
df_dates = [pd.to_datetime(df.iloc[2+i].values[0]) for i in list(range(0, len(df)-2, 3))]
df_ids = [int(df.iloc[3+i].values[0]) for i in list(range(0, len(df)-4, 3))]
df_scores = [int(df.iloc[4+i].values[0]) for i in list(range(0, len(df)-2, 3))]

# Create a DataFrame with the correct columns from the data
df = pd.DataFrame({'event_date': df_dates,
                  'id': df_ids,
                  'score':df_scores})
df.head()

Metrics are tracked across time, and this will track NPS month by month.

1. Create a numeric column in `df` called `yearmonth` that stores the year and the month of each row in the `yyyymm` format.

2. Create a column in `df` called `"category"` that assigns the category corresponding to its score to each row. Use the function `categorize` 

3. Create a new dataframe with the NPS for each month:

   1. Assign to `nps` the result of using `df.pivot_table()` with:
      - `yearmonth` as the index
      - `category` as a column
      - The string `size` as an aggregating function

   2. Create a new column in `nps` whose values are the sum of the values of each row to get the total number of responses for the month. This column should be called `total_responses`.

   3. Subtract detractors from the promoters and divide the result by the `total_responses` column. Assign the result of this computation to `nps`

   4. Multiply the `nps` column by 100, cast it as integer using the [`Series.astype()` method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.astype.html). Assign it back to `nps["nps"]`.

In [ ]:
df["yearmonth"] = df["event_date"].dt.strftime('%Y%m')

df["category"] = df["score"].apply(categorize)

nps = df.pivot_table(index="yearmonth", columns="category", aggfunc="size")
nps['total_responses']=nps.sum(axis=1)
nps['nps'] = (nps['Promoter'] - nps['Detractor'])/nps['total_responses']
nps['nps'] = (100*nps['nps']).astype(int)
nps

------
## ANALYZYING NPS

Note that `nps` as an integer, as this is how the net promoter score is usually presented.

Typically, when presenting a metric, it is done so in a dashboard, together with other relevant metrics. The following is a customer satisfaction dashboard taken fron [SlideModel.com](www.SlideModel.com)

![](images/slidemodel.jpg)

It is lacking chronological context. The NPS is 48; did it grow or did it decrease? How is it doing relative to the past few months?

Create a graph for `nps`:

In [ ]:
nps_graph = nps.reset_index()
plt.figure(figsize=(10,6))
chart = sns.lineplot(x='yearmonth',y='nps',data=nps_graph)
chart.set_xticks(nps_graph.index)
chart.set_xticklabels(nps_graph['yearmonth'], rotation=45)
chart.set_ylabel('NPS')
chart.set_title('NPS per Month')
plt.savefig('output_images/NPS_per_month.png', bbox_inches='tight')

Now suppose it is December 2020, the NPS is just above 30. We can immediately tell it is doing bad, at least in view of the last few months. What to do about this? This is where the "actionable" characteristic comes in.

If we want to improve NPS, we must improve customer satisfaction. The company can begin by surveying its customers to get feedback and gain some understanding as to what's driving this apparent insatisfaction. In fact, it's standard practice in NPS surveys to have a freeform text box where customers can share their feedback with us. Once we have this feedback, we can act upon it as see how it impacts our NPS; hopefully it will improve it.

It can also happen that the result of the survey shows that what led to this dissatisfaction was a recent change in the product. This is something to watch out for even before the survey is completed.

Also note how NPS ticks several, if not all, of the boxes of what makes a good metric: it is accurate, simple and intelligible, easy to drill down into (we know the score of every customer), it's dynamic, and so on.

Every metric has its pros and cons. It may seem like NPS is too simplistic a model to be useful, but it is in fact positively correlated with business growth. [Fred Reichheld](https://en.wikipedia.org/wiki/Fred_Reichheld), a business strategist, even called it "the one number you need to grow".

Cons include:

- Requiring feedback from customers for it to be actionable — and most customers don't give feedback.
- It doesn't account for the influence of each respondent, or the influence of each category of respondents. Perhaps unsatisfied customers are more likely to voice their unhappiness on social media than promoters are to voice how happy they are, for instance.

In December, the NPS was very low, relative to the past few months. But is it actually low? Perhaps it was the other months that were high. Gauging how good an NPS is requires taking into account other factors, like geographic location and industry.



![](images/retently_resized.png)

As we can see, net promoter scores vary greatly across industries — telecommunication companies (unlisted above) even average a non-positive NPS! In order to gain insight into how a NPS fares, we must take these factors into account.